In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
import numpy as np

In [2]:
df = pd.read_parquet("s3://projectdatagroup1/OmkarSample/SampleInput/TransformedOutput/part-00000-adc4b278-76eb-45ef-afe0-933481911f60-c000.snappy.parquet")

/home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/fsspec/registry.py:283: UserWarning: Your installed version of s3fs is very old and known to cause
severe performance issues, see also https://github.com/dask/dask/issues/10276

To fix, you should specify a lower version bound on s3fs, or
update the current installation.

  warnings.warn(s3_msg)


In [4]:
bins = [0, 6, 12, 18, 24]
labels = ['Night', 'Morning', 'Afternoon', 'Evening']
df['SchdDepTimeOfDay'] = pd.cut(df['SchdDepHour'], bins=bins, labels=labels, right=False)
df["IsWeekend"] = df["DayOfWeek"].apply(lambda x: 1 if x in [6, 7] else 0)

In [5]:
# Define features and target
X = df.drop("DelayCategory", axis=1)  # Features
y = df["DelayCategory"]  # Target

In [6]:
categorical_cols = ['Airline', 'Origin', 'Dest', 'OriginStateName', 'DestStateName','SchdDepTimeOfDay']
numerical_cols = [
    "SchdDepHour", "Year", "Quarter", "Month", "DayofMonth", "DayOfWeek",
    "TaxiOut", "WheelsOff", "WheelsOn", "TaxiIn", "SchdArrTime", "IsWeekend"
]

In [7]:
# Preprocessing pipeline
preprocessor = ColumnTransformer(
    transformers=[
        ("num", StandardScaler(), numerical_cols),  # Scale numerical features
        ("cat", OneHotEncoder(handle_unknown="ignore"), categorical_cols),  # One-hot encode categorical features
    ]
)

In [8]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestClassifier
from sklearn.impute import SimpleImputer
from sklearn.metrics import classification_report

In [9]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [10]:
# Apply preprocessing
X_train = preprocessor.fit_transform(X_train)
X_test = preprocessor.transform(X_test)

In [12]:
# Convert target to numerical labels
from sklearn.preprocessing import LabelEncoder

label_encoder = LabelEncoder()
y_train = label_encoder.fit_transform(y_train)
y_test = label_encoder.transform(y_test)

In [11]:
%pip install xgboost pandas scikit-learn

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.5/4.5 MB 67.1 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [13]:
from xgboost import XGBClassifier

from sklearn.metrics import accuracy_score, classification_report

/home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/xgboost/core.py:265: FutureWarning: Your system has an old version of glibc (< 2.28). We will stop supporting Linux distros with glibc older than 2.28 after **May 31, 2025**. Please upgrade to a recent Linux distro (with glibc 2.28+) to use future versions of XGBoost.
Note: You have installed the 'manylinux2014' variant of XGBoost. Certain features such as GPU algorithms or federated learning are not available. To use these features, please upgrade to a recent Linux distro with glibc 2.28+, and install the 'manylinux_2_28' variant.
  warnings.warn(


In [19]:
import joblib

# Save the trained model
# path = 's3://projectdatagroup1/OmkarSample/SampleInput/XGBoost_ModelOutput/'
joblib.dump(xgb, 'XGBoost.pkl')

# Load the model later
# xgb_loaded = joblib.load("xgboost_model.pkl")


['XGBoost.pkl']

In [20]:
import boto3
import joblib
import pickle
from xgboost import XGBClassifier


In [23]:
import joblib

# Save model
joblib.dump(xgb, "s3://projectdatagroup1/OmkarSample/SampleInput/XGBoost_ModelOutput/xgboost_model.pkl")

print("Model saved successfully!")

FileNotFoundError: [Errno 2] No such file or directory: 's3://projectdatagroup1/OmkarSample/SampleInput/XGBoost_ModelOutput/xgboost_model.pkl'

In [24]:
import boto3

session = boto3.Session()
credentials = session.get_credentials()

if credentials:
    print("AWS credentials found.")
else:
    print("AWS credentials NOT found. Run `aws configure` to set up.")


AWS credentials found.


In [ ]:
export AWS_ACCESS_KEY_ID="your-access-key"
export AWS_SECRET_ACCESS_KEY="your-secret-key"
export AWS_DEFAULT_REGION="your-region"


In [14]:
# Initialize the model
xgb = XGBClassifier(random_state=42, use_label_encoder=False, eval_metric='logloss')

# Train the model
xgb.fit(X_train, y_train)

# Predict on the test set
y_pred_xgb = xgb.predict(X_test)

# Evaluate accuracy
accuracy_xgb = accuracy_score(y_test, y_pred_xgb)
print(f"XGBoost Accuracy: {accuracy_xgb:.4f}")
print(classification_report(y_test, y_pred_xgb))

/home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/xgboost/core.py:158: UserWarning: [09:58:32] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)


XGBoost Accuracy: 0.9713
              precision    recall  f1-score   support

           0       0.90      0.88      0.89      1803
           1       0.91      0.88      0.89      5255
           2       0.98      1.00      0.99    123670
           3       0.93      0.84      0.88     17234
           4       0.46      0.14      0.21        79

    accuracy                           0.97    148041
   macro avg       0.84      0.75      0.77    148041
weighted avg       0.97      0.97      0.97    148041

